In [2]:
import pandas as pd
import numpy as np

In [3]:
# Loading original raw data
raw_data = pd.read_csv('Datasets_MS_Project/Temperature_change_on_land/Temperature_change_E_All_Data_(Normalized)/Environment_Temperature_change_E_All_Data_(Normalized).csv')
raw_data.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb0 in position 188: invalid start byte

There is an encoding error. Let's force reading with latin1 encoding which can tolerate 
those non-standard bytes.

In [4]:
raw_data = pd.read_csv('Datasets_MS_Project/Temperature_change_on_land/Temperature_change_E_All_Data_(Normalized)/Environment_Temperature_change_E_All_Data_(Normalized).csv', encoding = 'latin1')
raw_data.head()

,Area Code,Area Code (M49),Area,Months Code,Months,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,'004,Afghanistan,7001,January,7271,Temperature change,1961,1961,°c,0.745,E
1,2,'004,Afghanistan,7001,January,7271,Temperature change,1962,1962,°c,0.015,E
2,2,'004,Afghanistan,7001,January,7271,Temperature change,1963,1963,°c,2.706,E
3,2,'004,Afghanistan,7001,January,7271,Temperature change,1964,1964,°c,-5.250,E
4,2,'004,Afghanistan,7001,January,7271,Temperature change,1965,1965,°c,1.854,E


In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563040 entries, 0 to 563039
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Area Code        563040 non-null  int64  
 1   Area Code (M49)  563040 non-null  object 
 2   Area             563040 non-null  object 
 3   Months Code      563040 non-null  int64  
 4   Months           563040 non-null  object 
 5   Element Code     563040 non-null  int64  
 6   Element          563040 non-null  object 
 7   Year Code        563040 non-null  int64  
 8   Year             563040 non-null  int64  
 9   Unit             563040 non-null  object 
 10  Value            537452 non-null  float64
 11  Flag             563040 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 51.5+ MB


In [7]:
# Finding unique values of Element column
raw_data['Element'].unique()

array(['Temperature change', 'Standard Deviation'], dtype=object)

In [8]:
# Finding unique values of Months column
raw_data['Months'].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December',
       'Dec\x96Jan\x96Feb', 'Mar\x96Apr\x96May', 'Jun\x96Jul\x96Aug',
       'Sep\x96Oct\x96Nov', 'Meteorological year'], dtype=object)

We will just keep the data for the temperature change of three 3-month intervals 
and the meteorological year

In [ ]:
# Filter the rows
data_to_keep = ['Dec\x96Jan\x96Feb', 'Mar\x96Apr\x96May', 'Jun\x96Jul\x96Aug',
       'Sep\x96Oct\x96Nov', 'Meteorological year']
filtered_data = raw_data.loc[
    (raw_data['Element']=='Temperature change') &
    (raw_data['Months'].isin(data_to_keep))
]
filtered_data.head()

,Area Code,Area Code (M49),Area,Months Code,Months,Element Code,Element,Year Code,Year,Unit,Value,Flag
1512,2,'004,Afghanistan,7016,DecJanFeb,7271,Temperature change,1961,1961,°c,-0.763,E
1513,2,'004,Afghanistan,7016,DecJanFeb,7271,Temperature change,1962,1962,°c,0.984,E
1514,2,'004,Afghanistan,7016,DecJanFeb,7271,Temperature change,1963,1963,°c,1.939,E
1515,2,'004,Afghanistan,7016,DecJanFeb,7271,Temperature change,1964,1964,°c,-2.038,E
1516,2,'004,Afghanistan,7016,DecJanFeb,7271,Temperature change,1965,1965,°c,-0.638,E


In [11]:
# Confirm if the filter worked correctly
filtered_data['Months'].unique()

array(['Dec\x96Jan\x96Feb', 'Mar\x96Apr\x96May', 'Jun\x96Jul\x96Aug',
       'Sep\x96Oct\x96Nov', 'Meteorological year'], dtype=object)

In [12]:
# Reshaping data to the wide-format so that each of the 4 elements in the 'Months'
# column has its own separate column
pivoted_data = filtered_data.pivot_table(
    index = ['Area Code', 'Area', 'Year Code', 'Year'],
    columns = 'Months',
    values = 'Value'
)
pivoted_data.head()

Months                            DecJanFeb  JunJulAug  MarAprMay  \
Area Code Area    Year Code Year                                          
1         Armenia 1992      1992       -1.921       -0.893       -1.907   
                  1993      1993       -1.611       -0.106       -0.854   
                  1994      1994        0.265       -0.121        0.973   
                  1995      1995        0.383        0.890        1.305   
                  1996      1996        0.701        0.746        0.198   

Months                            Meteorological year  SepOctNov  
Area Code Area    Year Code Year                                    
1         Armenia 1992      1992               -1.305       -0.499  
                  1993      1993               -0.936       -1.173  
                  1994      1994                0.477        0.792  
                  1995      1995                0.694        0.197  
                  1996      1996                0.474        0.252

In [14]:
# Resetting row index
pivoted_data = pivoted_data.reset_index()
pivoted_data.head()

Months,Area Code,Area,Year Code,Year,DecJanFeb,JunJulAug,MarAprMay,Meteorological year,SepOctNov
0,1,Armenia,1992,1992,-1.921,-0.893,-1.907,-1.305,-0.499
1,1,Armenia,1993,1993,-1.611,-0.106,-0.854,-0.936,-1.173
2,1,Armenia,1994,1994,0.265,-0.121,0.973,0.477,0.792
3,1,Armenia,1995,1995,0.383,0.890,1.305,0.694,0.197
4,1,Armenia,1996,1996,0.701,0.746,0.198,0.474,0.252


In [15]:
# Setting the name of the column index to None
pivoted_data.columns.name = None
pivoted_data.head()

,Area Code,Area,Year Code,Year,DecJanFeb,JunJulAug,MarAprMay,Meteorological year,SepOctNov
0,1,Armenia,1992,1992,-1.921,-0.893,-1.907,-1.305,-0.499
1,1,Armenia,1993,1993,-1.611,-0.106,-0.854,-0.936,-1.173
2,1,Armenia,1994,1994,0.265,-0.121,0.973,0.477,0.792
3,1,Armenia,1995,1995,0.383,0.890,1.305,0.694,0.197
4,1,Armenia,1996,1996,0.701,0.746,0.198,0.474,0.252


In [16]:
# Renaming the columns
cleaned_data = pivoted_data.rename(
    columns = {
        'Area Code': 'area_code',
        'Area': 'area',
        'Year Code': 'year_code',
        'Year': 'year',
        'Dec\x96Jan\x96Feb': 'temp_change_Dec_Jan_Feb', 
        'Mar\x96Apr\x96May': 'temp_change_Mar_Apr_May', 
        'Jun\x96Jul\x96Aug': 'temp_change_Jun_Jul_Aug',
        'Sep\x96Oct\x96Nov': 'temp_change_Sep_Oct_Nov', 
        'Meteorological year': 'temp_change_meteorological_year'
    }
)
cleaned_data.head()

,area_code,area,year_code,year,temp_change_Dec_Jan_Feb,temp_change_Jun_Jul_Aug,temp_change_Mar_Apr_May,temp_change_meteorological_year,temp_change_Sep_Oct_Nov
0,1,Armenia,1992,1992,-1.921,-0.893,-1.907,-1.305,-0.499
1,1,Armenia,1993,1993,-1.611,-0.106,-0.854,-0.936,-1.173
2,1,Armenia,1994,1994,0.265,-0.121,0.973,0.477,0.792
3,1,Armenia,1995,1995,0.383,0.890,1.305,0.694,0.197
4,1,Armenia,1996,1996,0.701,0.746,0.198,0.474,0.252


In [19]:
# Exporting cleaned data to a csv file 
cleaned_data.to_csv('cleaned_datasets/temp_change_cleaned.csv', index=False, encoding='utf-8')